In [31]:
import pandas as pd
import numpy as np

from IPython.display import display, clear_output  
import PIL.Image
import cv2
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split

In [2]:
class EmotionRec(nn.Module):
    def __init__(self):
        super(EmotionRec, self).__init__()
        self.features=nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(32,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.25),
            nn.Conv2d(64,128,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(128,128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.25)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*6*6, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,4)
        )

    def forward(self,x):
        x=self.features(x)
        x=self.classifier(x)
        return x

In [3]:
class EmotionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32) / 255.0  
        image = image.reshape(48, 48, 1) 
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predictions = torch.max(outputs, 1)
            train_correct += (predictions == labels).sum().item()

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / len(train_loader.dataset)

        
        model.eval()
        val_loss = 0.0
        val_correct = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predictions = torch.max(outputs, 1)
                val_correct += (predictions == labels).sum().item()

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    return model

In [8]:
def main():
    df=pd.read_csv("fer2013.csv")

    emotions_to_keep = {0: 'Angry', 3: 'Happy', 4: 'Sad', 6: 'Calm'}
    df = df[df['emotion'].isin(emotions_to_keep.keys())]

    new_label_map = {old_label: new_label for new_label, old_label in enumerate(emotions_to_keep.keys())}

    labels = df['emotion'].map(new_label_map).values
    pixels = df['pixels'].apply(lambda x: np.array(x.split()).astype(int)).values

    X_train, X_val, y_train, y_val = train_test_split(pixels, labels, test_size=0.2, random_state=42)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    train_dataset = EmotionDataset(X_train, y_train, transform=transform)
    val_dataset = EmotionDataset(X_val, y_val, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = EmotionRec()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

    torch.save(trained_model.state_dict(), 'emotion_recognition_model.pth')

In [9]:
if __name__ == "__main__":
    main()

Epoch 1/10:
Train Loss: 1.2771, Train Acc: 0.4064
Val Loss: 1.0895, Val Acc: 0.5288
Epoch 2/10:
Train Loss: 1.0394, Train Acc: 0.5451
Val Loss: 0.9418, Val Acc: 0.5814
Epoch 3/10:
Train Loss: 0.9422, Train Acc: 0.5970
Val Loss: 0.9216, Val Acc: 0.6117
Epoch 4/10:
Train Loss: 0.8845, Train Acc: 0.6289
Val Loss: 0.8550, Val Acc: 0.6358
Epoch 5/10:
Train Loss: 0.8388, Train Acc: 0.6501
Val Loss: 0.8137, Val Acc: 0.6623
Epoch 6/10:
Train Loss: 0.7949, Train Acc: 0.6708
Val Loss: 0.8059, Val Acc: 0.6638
Epoch 7/10:
Train Loss: 0.7635, Train Acc: 0.6837
Val Loss: 0.7897, Val Acc: 0.6773
Epoch 8/10:
Train Loss: 0.7298, Train Acc: 0.7031
Val Loss: 0.7817, Val Acc: 0.6810
Epoch 9/10:
Train Loss: 0.7076, Train Acc: 0.7117
Val Loss: 0.7927, Val Acc: 0.6827
Epoch 10/10:
Train Loss: 0.6815, Train Acc: 0.7233
Val Loss: 0.7903, Val Acc: 0.6760


In [11]:
model = EmotionRec()
model.load_state_dict(torch.load('emotion_recognition_model.pth'))
model.eval()

EmotionRec(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Dropout(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4608, out_features=1024, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=1024, out_features=4, bias=True)
  )
)

In [14]:
emotion_dict = {0: "Angry", 1: "Happy", 2: "Sad", 3: "Calm"}

In [15]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [21]:
def emotion_recog(frame):
    
    cv2.ocl.setUseOpenCL(False)

  
    facecasc = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (255, 0, 255), 3)
        roi_gray = gray[y:y + h, x:x + w]

        cropped_img = cv2.resize(roi_gray, (48, 48))
        cropped_img = np.expand_dims(cropped_img, axis=2)
        cropped_img = transform(cropped_img).unsqueeze(0).to(device)

        with torch.no_grad():
            prediction = model(cropped_img)
            maxindex = int(torch.argmax(prediction))

        cv2.putText(frame, emotion_dict[maxindex], (x + 20, y - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return frame

In [22]:
def display_frame_in_notebook(frame):
    
    clear_output(wait=True)  
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
    pil_img = PIL.Image.fromarray(frame)  
    display(pil_img)  


In [29]:
def capture_and_identify_emotions():
    cap = cv2.VideoCapture(0)  

    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    try:
        while True:
            
            ret, frame = cap.read()

            if not ret:
                print("Error: Failed to capture image.")
                break

            
            frame_with_emotions = emotion_recog(frame)

            
            display_frame_in_notebook(frame_with_emotions)

            
            time.sleep(0.1)

    except KeyboardInterrupt:
        print("Stopped by user.")

    finally:
       
        cap.release()



In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [1]:
capture_and_identify_emotions()

NameError: name 'capture_and_identify_emotions' is not defined